In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

In [3]:
np.random.seed(42)
random.seed(42)

def generate_ab_test_data(group_size=1000):
    """
    Генерирует данные для AB-теста
    Group A: Исходный процесс
    Group B: Оптимизированный процесс
    """
    data = []
    
    # Базовые параметры для обеих групп
    start_date = datetime(2024, 1, 1)
    activities = [
        'App Download',
        'Account Registration', 
        'Phone Verification',
        'Email Verification',
        'ID Upload',
        'ID Verification',
        'Security Setup',
        'Tutorial Start',
        'Tutorial Complete',
        'First Transaction',
        'Dropoff'
    ]
    
    for group in ['A', 'B']:
        for case_id in range(1, group_size + 1):
            case_data = []
            current_time = start_date + timedelta(days=random.randint(0, 30))
            case_num = f"{group}_{case_id:04d}"
            
            # Все начинают с загрузки приложения
            case_data.append(create_event(case_num, 'App Download', current_time))
            current_time += timedelta(minutes=random.randint(5, 30))
            
            # Регистрация аккаунта
            case_data.append(create_event(case_num, 'Account Registration', current_time))
            current_time += timedelta(minutes=random.randint(3, 15))
            
            # Верификация телефона
            if random.random() < 0.95:  # 5% отваливаются здесь
                case_data.append(create_event(case_num, 'Phone Verification', current_time))
                current_time += timedelta(minutes=random.randint(2, 10))
                
                # Верификация email
                if random.random() < 0.93:  # Дополнительные 7% отваливаются
                    case_data.append(create_event(case_num, 'Email Verification', current_time))
                    current_time += timedelta(minutes=random.randint(2, 8))
                    
                    # Загрузка документа (различия между группами)
                    case_data.append(create_event(case_num, 'ID Upload', current_time))
                    
                    if group == 'A':
                        # Группа A: сложный процесс верификации
                        current_time += timedelta(hours=random.randint(2, 24))
                        if random.random() < 0.85:  # 15% не проходят верификацию
                            case_data.append(create_event(case_num, 'ID Verification', current_time))
                            current_time += timedelta(minutes=random.randint(10, 30))
                    else:
                        # Группа B: оптимизированный процесс
                        current_time += timedelta(hours=random.randint(1, 4))
                        if random.random() < 0.92:  # Улучшено на 7%
                            case_data.append(create_event(case_num, 'ID Verification', current_time))
                            current_time += timedelta(minutes=random.randint(5, 15))
                    
                    # Настройка безопасности
                    if 'ID Verification' in [e['concept:name'] for e in case_data]:
                        case_data.append(create_event(case_num, 'Security Setup', current_time))
                        
                        if group == 'A':
                            current_time += timedelta(minutes=random.randint(10, 25))
                        else:
                            current_time += timedelta(minutes=random.randint(5, 15))  # Оптимизировано
                        
                        # Обучение (различия между группами)
                        if random.random() < (0.70 if group == 'A' else 0.85):
                            case_data.append(create_event(case_num, 'Tutorial Start', current_time))
                            
                            if group == 'A':
                                current_time += timedelta(minutes=random.randint(15, 40))
                            else:
                                current_time += timedelta(minutes=random.randint(8, 20))  # Укороченный tutorial
                            
                            if random.random() < (0.80 if group == 'A' else 0.90):
                                case_data.append(create_event(case_num, 'Tutorial Complete', current_time))
                                current_time += timedelta(minutes=random.randint(5, 15))
                                
                                # Первая транзакция
                                if random.random() < (0.65 if group == 'A' else 0.78):
                                    case_data.append(create_event(case_num, 'First Transaction', current_time))
            
            # Если не дошли до конца - отмечаем как дропофф
            final_activity = case_data[-1]['concept:name']
            if final_activity != 'First Transaction':
                current_time += timedelta(minutes=random.randint(1, 10))
                case_data.append(create_event(case_num, 'Dropoff', current_time))
            
            data.extend(case_data)
    
    return pd.DataFrame(data)

In [5]:
def create_event(case_id, activity, timestamp):
    return {
        'case:concept:name': case_id,
        'concept:name': activity,
        'time:timestamp': timestamp,
        'group': case_id.split('_')[0]
    }

In [13]:
# Генерируем данные
print("Генерация данных для AB-теста...")
df_ab = generate_ab_test_data(1000)
print(f"Сгенерировано {len(df_ab)} событий")
print(f"Уникальных кейсов: {df_ab['case:concept:name'].nunique()}")
print(f"Распределение по группам:")
print(df_ab['group'].value_counts())

Генерация данных для AB-теста...
Сгенерировано 16644 событий
Уникальных кейсов: 2000
Распределение по группам:
group
B    8522
A    8122
Name: count, dtype: int64


In [17]:
df_ab.head(10)

,case:concept:name,concept:name,time:timestamp,group
0,A_0001,App Download,2024-01-18 00:00:00,A
1,A_0001,Account Registration,2024-01-18 00:28:00,A
2,A_0001,Phone Verification,2024-01-18 00:39:00,A
3,A_0001,Email Verification,2024-01-18 00:49:00,A
4,A_0001,ID Upload,2024-01-18 00:57:00,A
5,A_0001,Dropoff,2024-01-18 12:05:00,A
6,A_0002,App Download,2024-01-02 00:00:00,A
7,A_0002,Account Registration,2024-01-02 00:13:00,A
8,A_0002,Phone Verification,2024-01-02 00:22:00,A
9,A_0002,Email Verification,2024-01-02 00:29:00,A


In [19]:
df_ab.to_csv('ab_banking_event_log.csv', index=False)

In [21]:
df_ab.dtypes

case:concept:name            object
concept:name                 object
time:timestamp       datetime64[ns]
group                        object
dtype: object